In [1]:
source = 'online_retail.csv'

In [2]:
import pandas as pd

df = pd.read_csv(source)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,560915,22302,COFFEE MUG PEARS DESIGN,6,7/21/2011 18:19,2.55,13313.0,United Kingdom
1,560915,84755,COLOUR GLASS T-LIGHT HOLDER HANGING,16,7/21/2011 18:19,0.65,13313.0,United Kingdom
2,560915,23153,ZINC SWEETHEART WIRE LETTER RACK,4,7/21/2011 18:19,3.75,13313.0,United Kingdom
3,560915,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,7/21/2011 18:19,2.95,13313.0,United Kingdom
4,560916,23250,VINTAGE RED TRIM ENAMEL BOWL,24,7/21/2011 18:19,1.25,14911.0,EIRE
...,...,...,...,...,...,...,...,...
1510,561025,84987,SET OF 36 TEATIME PAPER DOILIES,1,7/24/2011 10:37,1.45,16241.0,United Kingdom
1511,561025,23306,SET OF 36 DOILIES PANTRY DESIGN,1,7/24/2011 10:37,1.45,16241.0,United Kingdom
1512,561025,22416,SET OF 36 DOILIES SPACEBOY DESIGN,1,7/24/2011 10:37,1.45,16241.0,United Kingdom
1513,561025,23298,SPOTTY BUNTING,2,7/24/2011 10:37,4.95,16241.0,United Kingdom


In [3]:
# 1: We skip cancelled orders
mask = df.iloc[:,0].str.startswith('C') != True

In [4]:
df.shape

(1515, 8)

In [5]:
df = df.dropna()

In [6]:
df.shape

(981, 8)

In [7]:
df = df.loc[mask]

In [8]:
# 2:  We built a dictionary with the InvoiceNo as key and as values the list of Descriptions associated with each invoic no
descriptions = {}
for row in df.itertuples():
    mask = df.iloc[:,0].str.startswith(row.InvoiceNo)
    descriptions[row.InvoiceNo] = df.loc[mask].Description.tolist()


In [9]:
# 2: as list of list
descriptions = []
invoice_nums = set(df.loc[:,'InvoiceNo'].tolist())

for num in invoice_nums:
    mask = df.iloc[:,0].str.startswith(num)
    descriptions.append(df.loc[mask].Description.tolist())


In [10]:
len(descriptions)

50

In [11]:
# 3: We define the matrix
desc_set = set()
for l in descriptions:
    for desc in l:
        desc_set.add(desc)

In [12]:
matrix = []
for l in descriptions:
    curr_row = []
    for item in desc_set:
        if item in l:
            curr_row.append(1)
        else:
            curr_row.append(0)
    matrix.append(curr_row)

In [ ]:
# We coulde have use a dictionary instead of the set to lower the time consumed

In [13]:
all_items = list(desc_set)
matrix_df = pd.DataFrame(data=matrix, columns=all_items)

In [14]:
matrix_df

,ENAMEL BREAD BIN CREAM,36 PENCILS TUBE RED RETROSPOT,GARAGE KEY FOB,TOY TIDY PINK POLKADOT,SET OF 4 KNICK KNACK TINS LONDON,FRENCH WC SIGN BLUE METAL,JAM CLOCK MAGNET,BUNDLE OF 3 ALPHABET EXERCISE BOOKS,VICTORIAN GLASS HANGING T-LIGHT,VINTAGE SNAP CARDS,...,BAG 125g SWIRLY MARBLES,SET OF 6 TEA TIME BAKING CASES,DO NOT TOUCH MY STUFF DOOR HANGER,ASSORTED COLOUR MINI CASES,SMALL RED RETROSPOT MUG IN BOX,PACK OF 12 PINK PAISLEY TISSUES,VINTAGE SNAKES & LADDERS,RED RETROSPOT MINI CASES,EDWARDIAN PARASOL BLACK,TRAVEL CARD WALLET SKULLS
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
7,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,0,1,...,1,1,0,1,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# 4: We can apply FP Growth
from mlxtend.frequent_patterns import fpgrowth
import timeit

fpgrowth(matrix_df, 0.1)

,support,itemsets
0,0.22,(283)
1,0.10,(452)
2,0.12,(554)
3,0.12,(437)
4,0.10,(521)
5,0.10,(172)
6,0.14,(61)
7,0.10,(393)
8,0.10,(163)
9,0.10,(77)


In [22]:
# 6: the confidence is the frequence of B in transactions containing A
# = sup(A,B)/sup(A). Where sup(A,B)= #{A,B}/|T|
print(matrix_df.columns[102]+"  -->  "+matrix_df.columns[104]) 

mask = (matrix_df.iloc[:,102] != 0) & (matrix_df.iloc[:, 104] != 0)
sup_a_b = matrix_df.loc[mask].shape[0] / matrix_df.shape[0]
sup_a  = matrix_df.loc[matrix_df.iloc[:,102] != 0].shape[0] / matrix_df.shape[0]
print(f'The confidence is: '+ str(sup_a_b/sup_a))

WOODEN PICTURE FRAME WHITE FINISH  -->  WHITE BELL HONEYCOMB PAPER 
The confidence is: 0.0


In [ ]:
# 7: Find the assoc. rules with confidence as metric and min_thresh =  0.85
from mlxtend.frequent_patterns import association_rules

ar = association_rules(fi, metric='confidence', min_threshold=0.85)

In [25]:
# 8: Experiments with Apriori algorithm
from mlxtend.frequent_patterns import apriori
import timeit

timeit.timeit(lambda: apriori(matrix_df, min_support=0.1), number=1)

0.014826900000002752

In [1]:
import json

with open("modified_coco.json") as f:
    images = json.load(f)
dataset = [ list(set(image["annotations"])) for image in images ] # remove duplicates
len(dataset)


5000

# COCO dataset handling

In [7]:
all_items = set()
for i in dataset:
    for j in i:
        all_items.add(j)

In [13]:
matrix_coco = []
all_items = sorted(list(all_items))
for l in dataset:
    curr_row = []
    for item in all_items:
        if item in l:
            curr_row.append(1)
        else:
            curr_row.append(0)
    matrix_coco.append(curr_row)

In [11]:
all_items_set = set()
for items in dataset:
    all_items_set.update(items)
all_items = sorted(list(all_items_set))
presence_matrix = [ [ int(item in image) for item in all_items ] for image in dataset ]
#df = pd.DataFrame(presence_matrix, columns=all_items_set)


In [14]:
presence_matrix == matrix_coco

True